# The 2D Ising Model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 100

from matplotlib import colors as mcolors        # Packages to define custom colormaps
from matplotlib.animation import FuncAnimation  # and animations

# from numba import njit       # Import this if you're using @njit to speed up your code
from tqdm import tqdm          # This is a very cool package that allows you to see a "progress bar" to know how far a loop has progressed

<div class="alert alert-block alert-info">
    
<b>Note: </b> You have now reached the point where not using `numba` might start to slow your code down quite a bit, meaning that you will miss many interesting results. Do try to start using `@njit` to make your code faster. </div> 

In [ ]:
# @njit                                     # Once you've written your code using only numpy arrays, you can choose to speed it up using this command
def oneMCS(spins, beta, h, energy, J=1):
    '''
    Perform a single Monte-Carlo sweep on a lattice of spins. The function accepts an array of spins, the 
    inverse temperature, and the external magnetic field. The total system energy is also accepted in order
    to compute the change in energy more efficiently.
    
    Parameters:
    -----------
    • spins  : A 2D numpy array of L x L elements storing the L**2 spins.
    • beta   : The inverse temperature at which to simulate the lattice.
    • h      : Value of the external magnetic field.
    • energy : The total energy of the supplied spin lattice.
    • J      : The coupling constant (by default, this is 1).
    
    Returns:
    --------
    The function returns one value:
    • spins  : The newly updated spin lattice. 
    • energy : The total energy of the newly updated spin lattice.
    '''
    
    L = np.shape(spins)[0]                # Linear dimension of our square lattice
    N = L**2                              # Number of spins
    
    ## In the lines that follow use the algorithm given in the assignment to simulate one Monte Carlo sweep.
    ## It should still take roughly 15 to 20 lines of code.

    return spins, energy

In [ ]:
def simulate(L, beta, h, J=1, n_mcs = 10_000):
    '''
    Simulate a number of Monte-Carlo sweeps on a lattice of some size L.
    
    Parameters:
    -----------
    • L    : The side-length of the square lattice.
    • beta : The inverse temperature at which to simulate the lattice.
    • h    : Value of the external magnetic field.
    • J    : The coupling constant (by default, this is 1).
    • n_mcs: Number of Monte-Carlo sweeps. (Default = 10_000.)
    
    Returns:
    --------
    The function returns one value:
    • systemEnergy  : An array of the system's energy as a function of Monte-Carlo step.
    • m             : An array of the system's magnetisation as a function of Monte-Carlo step.
    '''
    
    spins = np.ones(shape=(L,L))
    N=L**2
    
    systemEnergy = np.zeros(n_mcs)   # Array to store system energies
    m = np.zeros_like(systemEnergy)
    
    ## In the lines that follow, complete the simulate function to run `n_mcs` sweeps
    ## and store the energy and magnetisation in the arrays provided above.
    ## It should take roughly 20 lines of code or so.
    

    return systemEnergy, m

<div class="alert alert-block alert-info">
    
<b>Note: </b>The `animate` function below is reasonably complicated,  but the good news is that you don't need to understand how it works to use it (although it'd be great if you did!). This function accepts an initial 2D spin configuration `spins` and an (inverse) temperature `beta`, and creates an animation of the simulation. Run the cell immediately after this one with the values of `beta` suggested in the assignment.
 </div> 

<div class="alert alert-block alert-success">

Do you see evidence of a temperature below which magnetic "domains" start to appear in the animation? Also, estimate roughly how many Monte-Carlo steps-per-spin the system needs to equilibriate. You can discard this number of `mcs` before you start taking averages in the later parts of this exercise.
 
You are _not_ expected to submit a video of the formation of such domains. You have been asked to include snapshots of a typical configuration in each case. Please insert them in your assignment at the right place, and with a sufficiently high `dpi` so that the image is clear when expanded. As a reminder, you can save an image by using the `plt.savefig("./name.png", dpi=300)` command before `plt.show()`.
</div>

In [ ]:
############### IGNORE THIS CELL IS SKIP STRAIGHT TO THE NEXT ONE ########################################################################################

#**** If you're here because your animation isn't working, the problem is most likely with your `oneMCS` function, and not with this one.  **************#
#**** Make sure the function returns two quantities, a number (the energy) and a 2D array (the new spin configuration).                    **************#

def animate(spins, beta, h=0, step=1, save_animation=False, save_name='video.mp4', save_dpi=300, save_fps=100):
    '''
    Function to return an imshow animation for the 2D Ising chain. It looks complicated because it is. But you 
    don't need to understand it, only use it. Notice how (as usual) I have defined many default options that 
    can be used to download the animation as a video.
    
    This function accepts an initial 2D spin configuration `spins` and an (inverse) temperature `beta`, and 
    produces an animation of the time-evolution of the system (in terms of Monte-Carlo steps).
    
    Parameters:
    -----------
    • spins          : A 2D numpy array of L x L elements storing the L**2 spins.
    • beta           : The inverse temperature at which to simulate the lattice.
    • h              : Value of the external magnetic field. (Default is 0.)
    • step           : Show every `step` Monte-Carlo steps in the animation. (Default is 1.)
    • save_animation : Save the animation as an .mp4 file. (Default is `video.mp4`.)
    • save_dpi       : Dots per inch of saved animation. (Default is 300.)
    • save_fps       : Frames per second of saved animation. (Default is 100.)
    • J              : The coupling constant (by default, this is 1).
    
    Returns:
    --------
    The function returns one value:
    • ani : A FuncAnimation instance, so that the animation can be used outside this function.
    '''
    
    fig, axes = plt.subplots(ncols=3, figsize=(10,3), gridspec_kw={'width_ratios': [1, 1, 1]})  # Define a figure and an axes. axes work just like plt, i.e. you can just do axes[0].plot, etc.
    
    ######## SETTING UP THE AESTHETICS OF THE PLOT ###########################################
    
    def plotsetup():
        axes[0].set_aspect('equal'); axes[0].tick_params(top=False, bottom=False, left=False, right=False,  # Remove all axes (they're unnecessary here)
                                                       labelleft=False, labelbottom=False)
        
        axes[1].set_ylabel(r"$m=\langle M \rangle/N$"); axes[1].set_xlabel(r"Step")
        axes[2].set_ylabel(r"$\langle E \rangle/N$"); axes[2].set_xlabel(r"Step")
        
        plt.tight_layout()

    colors=['tomato', 'royalblue']   # Colours for the spins
    linewidth = 0.00                 # The grid's linewidth; I've set this to zero since for larger lattices 
                                     # this made it hard to see patterns. You can change it to 0.01 if you wish
    
    cmap = mcolors.ListedColormap(colors) # Making our list of colours into a colourmap
    bounds=[-1,0,1]                       # whose range goes from -1 to 1 in integer steps
    
    ############################################################################################
    
    L = len(spins); N = L**2; J=1; # Parameters of the simulation
    
    ###### Setting up the plots for the initial configuration ##################################
    
    initialEnergy = 0; initialM = 0 
    
    for i in range(L):
        for j in range(L):
            initialEnergy += -(J/2)*spins[i][j]*(spins[(i+1)%L][j] + spins[i][(j+1)%L] + spins[(i-1)][j] + spins[i][(j-1)]) - h*spins[i][j]    # Set the initial system energy
            initialM += spins[i][j]/N
    
    #### Array to store data per frame
    mdata = []; edata = []; framedata = []; spinHistory = []
    mdata.append(initialM); edata.append(initialEnergy); framedata.append(0*step); spinHistory.append(spins)
    
    ############################################################################################
    
    def init():          # Initialise the whole plot 
        plotsetup()
        return axes

    def animate(frame):  # For frame number `frame',

        frameSpins, frameEnergy = oneMCS(energy=edata[-1], beta=beta, h=h, spins=spinHistory[-1], J=J) # Run `oneMCS` and get the energy and new configuration

        # Store these values in new arrays
        spinHistory.append(frameSpins); mdata.append(np.mean(spinHistory[-1])); edata.append(frameEnergy); framedata.append(frame*step)
        
        img   = axes[0].pcolormesh(spins, edgecolors='k', linewidth=linewidth, cmap=cmap)    
        mplot = axes[1].scatter(framedata,np.array(mdata),color='firebrick', marker='x')
        eplot = axes[2].scatter(framedata,np.array(edata)/N,color='darkgoldenrod', marker='x')

        axes[0].set_aspect('equal')
        plotsetup()
        
        return img, mplot, eplot,
    
    # Most important line, this is what actually handles the animation by calling the `animate' function with a frame number (integer)
    ani = FuncAnimation(fig, animate, init_func=init, blit=True, frames=1000, interval=10, repeat=True)   # Code to create animations
    
    if(save_animation):                                  # If you wish to save the animation, do so with the dpi and fps set here 
        ani.save(save_name, dpi=save_dpi, fps=save_fps)

    return ani                                           # When calling this FuncAnimation function from with a code, all the information that update the window 
                                                         # are attributes of the object ani. If you do not keep a reference to it around, then ani is garbage collected 
                                                         # all information about the graphs disappears when calling from within a function. You don't need this when you call 
                                                         # aren't calling the animation from within a function.

In [ ]:
%matplotlib notebook

initial_spins =  # <--- Set an initial configuration of spins pointing randomly up or down, with size (say) 32 x 32. 

animate(initial_spins, beta = ) # <--- Complete this by setting a value of beta corresponding to a high temperature

In [ ]:
%matplotlib notebook

initial_spins = # <--- Set an initial configuration of spins pointing randomly up or down, with size (say) 32 x 32. 

animate(initial_spins, beta = ) # <--- Complete this by setting a value of beta corresponding to a low temperature

<div class="alert alert-block alert-info">

<b>Note:</b> My implementation of this function takes around 10 minutes to run without `numba` for even small lattices `Ls=[4,8,16]`, and `n_mcs=10_000`. The same function runs in **6s** when using `numba`.  For interesting results you should do this for larger lattices, and for `n_mcs` greater than `5_000`, meaning that you should really get your code to work with `@njit`.
    
</div>

In [ ]:
def comparingLattices(Ls=[4,8,16], colors=['navy', 'darkgreen', 'firebrick'], markers=['x', '.', '1']):
    '''
    Simulate square lattices of different side-lengths and produce graphs of the average energy per spin,
    average magnetisation per spin, specific heat and magnetic susceptibility.
    
    Parameters:
    -----------
    • Ls          : A list of different lattice lengths. (Default is [4,8,16].)
    • colors      : A list of colours to plot results for each lattice.  (Default is ['navy', 'darkgreen', 'firebrick'].)
    • markers     : A set of markers to plot results for each lattice. (Default is ['x', '.', '1'].)
    
    Returns:
    --------
    NoneType (A plot).
    '''
        
    if(len(Ls)>len(markers) or len(Ls)>len(colors)):
        raise ValueError("The `markers` and `colors` arrays should be atleast as long as the `Ls` array!")
    
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,8))  # Create a composite plot with 1 row and two columns.
                                                                # Each individual plot can be called using axes[0] and axes[1]
                                                                # If we had more than one row, axes would be 2D. i.e. we could 
                                                                # call the individual plots using axes[0][0] etc.
    # Parameters
    J = 1
    h = 0
    ############

    betas = np.linspace(0.3,0.7,20)    # <--- Complete this by choosing an appropriate range for beta
    
    avE   =                            # <--- Complete this by appropriately initialising these arrays
    avEsq =                            # <--- Complete this by appropriately initialising these arrays
    
    avM   =                            # <--- Complete this by appropriately initialising these arrays
    avMsq =                            # <--- Complete this by appropriately initialising these arrays
    
    for Li in tqdm(range(len(Ls))):                             # Loop over Ls. The `tqdm` function creates a progress-bar that auto-updates.
        L = Ls[Li]; N = L**2
        
        ## In the next ten lines, simulate the system for each value of `beta` in the array `betas`,
        ## and store the average energy, the average squared energy, the average (absolute) magnetisation, and the 
        ## average squared magnetisation (over mcs) for each value of `beta`. Make sure for each `beta` that you wait 
        ## for the system to reach equilibrium before taking the average.

        
        axes[0][0].scatter( , , color=colors[Li], marker=markers[Li], label=r"$(L="+str(L)+")$") # <--- Complete this by plotting the appropriate numerical quantities
        axes[0][0].set_ylabel(r"$\langle E \rangle$ (per spin)")
        axes[0][0].set_xlabel(r"$\beta$")
        axes[0][0].legend()

        axes[0][1].scatter( , , color=colors[Li], marker=markers[Li], label=r"$(L="+str(L)+")$") # <--- Complete this by plotting the appropriate numerical quantities
        axes[0][1].set_ylabel(r"$c_V$")
        axes[0][1].set_xlabel(r"$\beta$")
        axes[0][1].legend()
        
        axes[1][0].scatter( , , color=colors[Li], marker=markers[Li], label=r"$(L="+str(L)+")$") # <--- Complete this by plotting the appropriate numerical quantities
        axes[1][0].set_ylabel(r"$m = \langle M \rangle/N$")
        axes[1][0].set_xlabel(r"$\beta$")
        axes[1][0].legend()

        axes[1][1].scatter( , , color=colors[Li], marker=markers[Li], label=r"$(L="+str(L)+")$") # <--- Complete this by plotting the appropriate numerical quantities
        axes[1][1].set_ylabel(r"$\chi$")
        axes[1][1].set_xlabel(r"$\beta$")
        axes[1][1].legend()

In [ ]:
%%time
# The default values should take around 10 minutes to run without Numba, and 6 seconds to run with it.

comparingLattices()